<h3>Segmenting Submittal Part 1</h3>

In [7]:
# read the Wiki page into a dataframe
import pandas as pd

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [8]:
# Get names of indexes for which column Borough is marked as Not Assigned
indexNames = df[ df['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [9]:
# Where multiple neighbourhoods exist per postcode, aggregate them to a single row
df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [10]:
# where a neighbourhood is not assigned, make the Borough equal to the Neighbourhood
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']

In [11]:
df1.shape

(103, 2)

In [12]:
df1.head(10)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
M1E,Scarborough,"West Hill,Guildwood,Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"


<h3>Segmenting Submittal Part #2- Create new dataframe with the geo coordinates</h3>

In [13]:
df2 = pd.read_csv (r'http://cocl.us/Geospatial_data')

In [14]:
print (df2)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24          M2R  43.782736 -79.442259
25          

In [15]:
df3 = pd.merge(df1,df2, how='left', left_on='Postcode', right_on='Postal Code')
df3

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek,Port Union,Rouge Hill",M1C,43.784535,-79.160497
2,Scarborough,"West Hill,Guildwood,Morningside",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,Scarborough,"Kennedy Park,Ionview,East Birchmount Park",M1K,43.727929,-79.262029
7,Scarborough,"Golden Mile,Oakridge,Clairlea",M1L,43.711112,-79.284577
8,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",M1M,43.716316,-79.239476
9,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


<h3>Segmenting Submittal Part #3- Analysis of Venues in the North York Borough of Toronto<h3>

In [16]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [17]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'],df3['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
north_york = df3[df3['Borough'] == 'North York'].reset_index(drop=True)
north_york.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,North York,Hillcrest Village,M2H,43.803762,-79.363452
1,North York,"Fairview,Henry Farm,Oriole",M2J,43.778517,-79.346556
2,North York,Bayview Village,M2K,43.786947,-79.385975
3,North York,"York Mills,Silver Hills",M2L,43.757490,-79.374714
4,North York,"Newtonbrook,Willowdale",M2M,43.789053,-79.408493


In [20]:
address = 'North York'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [21]:
# create map of Toronto using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(north_york['Latitude'], north_york['Longitude'], north_york['Borough'],north_york['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

In [22]:
CLIENT_ID = 'MKIX5GO0RCMABWJ2HYMBTLBHAMHSPZLQTVSXOEH24SA5BHG4' # your Foursquare ID
CLIENT_SECRET = 'GNFV4N01EC4LK2EJEMVD25J2UNKCR3COGSBNZ4LWO30JFLVX' # your Foursquare Secret
VERSION = '20180602' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MKIX5GO0RCMABWJ2HYMBTLBHAMHSPZLQTVSXOEH24SA5BHG4
CLIENT_SECRET:GNFV4N01EC4LK2EJEMVD25J2UNKCR3COGSBNZ4LWO30JFLVX


In [23]:
north_york.loc[0, 'Neighbourhood']

'Hillcrest Village'

In [24]:
neighborhood_latitude = north_york.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


In [25]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=MKIX5GO0RCMABWJ2HYMBTLBHAMHSPZLQTVSXOEH24SA5BHG4&client_secret=GNFV4N01EC4LK2EJEMVD25J2UNKCR3COGSBNZ4LWO30JFLVX&v=20180602&ll=43.8037622,-79.3634517&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e38ca3c6d8c56001b95da7d'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.808262204500004,
    'lng': -79.3572281853783},
   'sw': {'lat': 43.7992621955, 'lng': -79.3696752146217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad9dce6f964a520651b21e3',
       'name': "Eagle's Nest Golf Club",
       'location': {'address': '10000 Dufferin Rd',
        'lat': 43.805454826002794,
        'lng': -79.36418592243415,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
          'lng': -79.36418592243415}],
        'distance': 197,
        'cc': 'CA',
        'city': 'Toronto

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# type your answer here

north_york_venues = getNearbyVenues(names=north_york['Neighbourhood'],
                                   latitudes=north_york['Latitude'],
                                   longitudes=north_york['Longitude']
                                  )



Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
York Mills,Silver Hills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Downsview North,Bathurst Manor,Wilson Heights
York University,Northwood Park
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Lawrence Manor East,Bedford Park
Lawrence Heights,Lawrence Manor
Glencairn
Downsview,Upwood Park,North Park
Humber Summit
Emery,Humberlea


In [32]:
print(north_york_venues.shape)
north_york_venues.head()

(234, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [33]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bayview Village,4,4,4,4,4,4
"CFB Toronto,Downsview East",4,4,4,4,4,4
Don Mills North,5,5,5,5,5,5
Downsview Central,2,2,2,2,2,2
"Downsview North,Bathurst Manor,Wilson Heights",19,19,19,19,19,19
Downsview Northwest,4,4,4,4,4,4
Downsview West,3,3,3,3,3,3
"Downsview,Upwood Park,North Park",3,3,3,3,3,3
"Emery,Humberlea",2,2,2,2,2,2


In [34]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 105 uniques categories.


In [35]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()
                


,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Fairview,Henry Farm,Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [36]:
north_york_onehot.shape

(234, 106)

In [37]:
north_york_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CFB Toronto,Downsview East",0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Don Mills North,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Downsview Central,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Downsview North,Bathurst Manor,Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.00,...,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000
5,Downsview Northwest,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview West,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Downsview,Upwood Park,North Park",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Emery,Humberlea",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Fairview,Henry Farm,Oriole",0.015873,0.00,0.015873,0.000000,0.015873,0.000000,0.031746,0.015873,0.00,...,0.000000,0.031746,0.015873,0.015873,0.015873,0.015873,0.000000,0.000000,0.015873,0.031746


In [38]:
north_york_grouped.shape

(24, 106)

In [39]:
num_top_venues = 5

for hood in north_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----CFB Toronto,Downsview East----
               venue  freq
0            Airport  0.25
1               Park  0.25
2  Other Repair Shop  0.25
3        Snack Place  0.25
4             Lounge  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3   Japanese Restaurant   0.2
4      Basketball Court   0.2


----Downsview Central----
               venue  freq
0     Baseball Field   0.5
1         Food Truck   0.5
2  Accessories Store   0.0
3     Massage Studio   0.0
4           Pharmacy   0.0


----Downsview North,Bathurst Manor,Wilson Heights----
                venue  freq
0         Coffee Shop  0.11
1         Pizza Place  0.05
2  Frozen Yogurt Shop  0.05
3            Pharmacy  0.05
4          

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_york_grouped['Neighborhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,"CFB Toronto,Downsview East",Park,Snack Place,Airport,Other Repair Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,Downsview Central,Food Truck,Baseball Field,Women's Store,Electronics Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,"Downsview North,Bathurst Manor,Wilson Heights",Coffee Shop,Fried Chicken Joint,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Ice Cream Shop,Diner,Restaurant,Middle Eastern Restaurant


In [42]:
# set number of clusters
kclusters = 5

north_york_grouped_clustering = north_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 4, 1, 1], dtype=int32)

In [43]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,"CFB Toronto,Downsview East",Park,Snack Place,Airport,Other Repair Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,Downsview Central,Food Truck,Baseball Field,Women's Store,Electronics Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,"Downsview North,Bathurst Manor,Wilson Heights",Coffee Shop,Fried Chicken Joint,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Ice Cream Shop,Diner,Restaurant,Middle Eastern Restaurant


In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [50]:
north_york_merged = north_york

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

north_york_merged.head() # check the last columns!

,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Hillcrest Village,M2H,43.803762,-79.363452,1,Golf Course,Dog Run,Pool,Mediterranean Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
1,North York,"Fairview,Henry Farm,Oriole",M2J,43.778517,-79.346556,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Tea Room,Juice Bar,Food Court,Women's Store,Bakery,Boutique
2,North York,Bayview Village,M2K,43.786947,-79.385975,1,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,North York,"York Mills,Silver Hills",M2L,43.757490,-79.374714,0,Cafeteria,Women's Store,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,North York,"Newtonbrook,Willowdale",M2M,43.789053,-79.408493,3,Piano Bar,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighbourhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        # color=rainbow[cluster-1],
        fill=True,
        # fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 2, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Humber Summit,2,Empanada Restaurant,Women's Store,Dog Run,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


<h3>The top venues for the North York Borough of Toronto were identified and mapped.  Four square did not appear to have enough data to make solid recommendations.  Further analysis and data is needed </h3>